# EXTRACT

### CitiBike Data Extraction from Online Database to Analyze the Gender Perspective of City Bike Usage 

In [46]:
import pandas as pd
import requests 
import zipfile
import io

In [48]:
# URLs for June 2013 and June 2019 data (Citi Bike archive)
urls = {
    "2013": "https://s3.amazonaws.com/tripdata/2013-citibike-tripdata.zip",
    "2019": "https://s3.amazonaws.com/tripdata/2019-citibike-tripdata.zip"
}

In [19]:
# fucntion to download and save the 2013 and 2019 ZIP file 
# download the file in chunck to prevent memory overload 
# comment out the below funtion to prevent memeory overload 

"""def download_zip(url, filename):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
                print(f'Downloaded: {filename}')
    else:
        print(f'Failed to download {filename}')

# download both files
for year, url in urls.items():
    download_zip(url, f"{year}-citibike-tripdata.zip")"""

'def download_zip(url, filename):\n    response = requests.get(url, stream=True)\n    if response.status_code == 200:\n        with open(filename, \'wb\') as file:\n            for chunk in response.iter_content(chunk_size=1024):\n                file.write(chunk)\n                print(f\'Downloaded: {filename}\')\n    else:\n        print(f\'Failed to download {filename}\')\n\n# download both files\nfor year, url in urls.items():\n    download_zip(url, f"{year}-citibike-tripdata.zip")'

### Analyzing the ZIP file and Checking which CSV File to Extract 

In [20]:
# Function to list the files inside the ZIP file without extracting them
def list_files_in_zip(zip_filename):
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        # List all the contents in the ZIP file
        file_list = zip_ref.namelist()
        return file_list

zip_files = ["2013-citibike-tripdata.zip", "2019-citibike-tripdata.zip"]

# Iterate over the ZIP files and print the file names inside
for zip_filename in zip_files:
    print(f"Files in {zip_filename}:")
    file_names = list_files_in_zip(zip_filename)
    for file_name in file_names:
        print(file_name)
    print('--------------------')  

Files in 2013-citibike-tripdata.zip:
2013-citibike-tripdata/
2013-citibike-tripdata/4_April/
2013-citibike-tripdata/12_December/
2013-citibike-tripdata/.DS_Store
__MACOSX/2013-citibike-tripdata/._.DS_Store
2013-citibike-tripdata/201309-citibike-tripdata.csv
__MACOSX/2013-citibike-tripdata/._201309-citibike-tripdata.csv
2013-citibike-tripdata/11_November/
2013-citibike-tripdata/7_July/
2013-citibike-tripdata/201311-citibike-tripdata.csv
__MACOSX/2013-citibike-tripdata/._201311-citibike-tripdata.csv
2013-citibike-tripdata/201307-citibike-tripdata.csv
__MACOSX/2013-citibike-tripdata/._201307-citibike-tripdata.csv
2013-citibike-tripdata/10_October/
2013-citibike-tripdata/9_September/
2013-citibike-tripdata/8_August/
2013-citibike-tripdata/6_June/
2013-citibike-tripdata/3_March/
2013-citibike-tripdata/201308-citibike-tripdata.csv
__MACOSX/2013-citibike-tripdata/._201308-citibike-tripdata.csv
2013-citibike-tripdata/1_January/
2013-citibike-tripdata/201306-citibike-tripdata.csv
__MACOSX/2013-

### Merging the June 2019 CSV Files 

In [40]:
# Merge the 2019 June csv files 
zip_file = "2019-citibike-tripdata.zip"
csv_files_2019 = {
    "2019_1": "2019-citibike-tripdata/6_June/201906-citibike-tripdata_1.csv",
    "2019_2": "2019-citibike-tripdata/6_June/201906-citibike-tripdata_2.csv",
    "2019_3": "2019-citibike-tripdata/6_June/201906-citibike-tripdata_3.csv"
}

# Function to extract and read CSV file from ZIP
def read_csv_from_zip(zip_filename, csv_filename):
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        with zip_ref.open(csv_filename) as file:
            return pd.read_csv(file)

# Read CSV files and store them in a list
dfs_june_2019 = []
for key, csv_file in csv_files_2019.items():
    df = read_csv_from_zip(zip_file, csv_file)
    dfs_june_2019.append(df)

# Concatenate all DataFrames into one
june_merged_df_2019 = pd.concat(dfs_june_2019, ignore_index=True)

# Save the merged DataFrame to a CSV file
june_merged_df_2019.to_csv("2019_citibike_june_tripdata_merged.csv", index=False)

# Check the first few rows of the merged DataFrame
print(june_merged_df_2019.head())

   tripduration                 starttime                  stoptime  \
0           330  2019-06-01 00:00:01.5000  2019-06-01 00:05:31.7600   
1           830  2019-06-01 00:00:04.2400  2019-06-01 00:13:55.1470   
2           380  2019-06-01 00:00:06.0190  2019-06-01 00:06:26.7790   
3          1155  2019-06-01 00:00:06.7760  2019-06-01 00:19:22.5380   
4          1055  2019-06-01 00:00:07.5200  2019-06-01 00:17:42.5580   

   start station id       start station name  start station latitude  \
0              3602           31 Ave & 34 St               40.763154   
1              3054  Greene Ave & Throop Ave               40.689493   
2               229           Great Jones St               40.727434   
3              3771  McKibbin St & Bogart St               40.706237   
4               441          E 52 St & 2 Ave               40.756014   

   start station longitude  end station id       end station name  \
0               -73.920827            3570         35 Ave & 37 St   
1 

In [41]:
# Calculate the percentage of null values for each column in the merged DataFrame
null_percentage = june_merged_df_2019.isnull().mean() * 100
print(null_percentage)

tripduration               0.0
starttime                  0.0
stoptime                   0.0
start station id           0.0
start station name         0.0
start station latitude     0.0
start station longitude    0.0
end station id             0.0
end station name           0.0
end station latitude       0.0
end station longitude      0.0
bikeid                     0.0
usertype                   0.0
birth year                 0.0
gender                     0.0
dtype: float64


### Extracrting the June 2013 CSV File for cleaning up 

In [42]:
# extract read the 2013 June csv file from the zip and clean it  
zip_file_2013 = "2013-citibike-tripdata.zip"
csv_file_2013 = "2013-citibike-tripdata/6_June/201306-citibike-tripdata_1.csv"

# Function to read a CSV file from a ZIP archive
def read_csv_from_zip(zip_filename, csv_filename):
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        with zip_ref.open(csv_filename) as file:
            df = pd.read_csv(file)
            return df

# Extract the 2013 June data
df_2013_june = read_csv_from_zip(zip_file_2013, csv_file_2013)
df_2013_june.to_csv("2013_citibike_june_tripdata.csv", index=False)
print(df_2013_june.head())

   tripduration            starttime             stoptime  start station id  \
0           695  2013-06-01 00:00:01  2013-06-01 00:11:36               444   
1           693  2013-06-01 00:00:08  2013-06-01 00:11:41               444   
2          2059  2013-06-01 00:00:44  2013-06-01 00:35:03               406   
3           123  2013-06-01 00:01:04  2013-06-01 00:03:07               475   
4          1521  2013-06-01 00:01:22  2013-06-01 00:26:43              2008   

       start station name  start station latitude  start station longitude  \
0      Broadway & W 24 St               40.742354               -73.989151   
1      Broadway & W 24 St               40.742354               -73.989151   
2  Hicks St & Montague St               40.695128               -73.995951   
3     E 15 St & Irving Pl               40.735243               -73.987586   
4   Little West St & 1 Pl               40.705693               -74.016777   

   end station id        end station name  end station l

In [43]:
# Calculate the percentage of null values in June 2013 data 
null_percentage = df_2013_june.isnull().mean() * 100
print(null_percentage)

tripduration                0.000000
starttime                   0.000000
stoptime                    0.000000
start station id            0.000000
start station name          0.000000
start station latitude      0.000000
start station longitude     0.000000
end station id              3.126001
end station name            3.126001
end station latitude        3.126001
end station longitude       3.126001
bikeid                      0.000000
usertype                    0.000000
birth year                 41.599403
gender                      0.000000
dtype: float64


#### Due to the presence of numerous null values in the June 2013 dataset, which could hinder analysis, the June 2016 dataset will be downloaded and extracted to provide more comprehensive data.

In [50]:
"""urls = {
    "2016": "https://s3.amazonaws.com/tripdata/2016-citibike-tripdata.zip"
}
def download_zip(url, filename):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
                print(f'Downloaded: {filename}')
    else:
        print(f'Failed to download {filename}')

for year, url in urls.items():
    download_zip(url, f"{year}-citibike-tripdata.zip")"""

'urls = {\n    "2016": "https://s3.amazonaws.com/tripdata/2016-citibike-tripdata.zip"\n}\ndef download_zip(url, filename):\n    response = requests.get(url, stream=True)\n    if response.status_code == 200:\n        with open(filename, \'wb\') as file:\n            for chunk in response.iter_content(chunk_size=1024):\n                file.write(chunk)\n                print(f\'Downloaded: {filename}\')\n    else:\n        print(f\'Failed to download {filename}\')\n\nfor year, url in urls.items():\n    download_zip(url, f"{year}-citibike-tripdata.zip")'

In [52]:
# Function to list the files inside the ZIP file without extracting them
def list_files_in_zip(zip_filename):
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        # List all the contents in the ZIP file
        file_list = zip_ref.namelist()
        return file_list

zip_files = ["2016-citibike-tripdata.zip"]

# Iterate over the ZIP files and print the file names inside
for zip_filename in zip_files:
    print(f"Files in {zip_filename}:")
    file_names = list_files_in_zip(zip_filename)
    for file_name in file_names:
        print(file_name)
    print('--------------------')  

Files in 2016-citibike-tripdata.zip:
2016-citibike-tripdata/
2016-citibike-tripdata/4_April/
2016-citibike-tripdata/12_December/
2016-citibike-tripdata/.DS_Store
__MACOSX/2016-citibike-tripdata/._.DS_Store
2016-citibike-tripdata/11_November/
2016-citibike-tripdata/7_July/
2016-citibike-tripdata/10_October/
2016-citibike-tripdata/9_September/
2016-citibike-tripdata/8_August/
2016-citibike-tripdata/6_June/
2016-citibike-tripdata/3_March/
2016-citibike-tripdata/1_January/
2016-citibike-tripdata/2_February/
2016-citibike-tripdata/5_May/
2016-citibike-tripdata/4_April/201604-citibike-tripdata_1.csv
2016-citibike-tripdata/4_April/201604-citibike-tripdata_2.csv
2016-citibike-tripdata/12_December/201612-citibike-tripdata_1.csv
2016-citibike-tripdata/11_November/201611-citibike-tripdata_2.csv
2016-citibike-tripdata/11_November/201611-citibike-tripdata_1.csv
2016-citibike-tripdata/7_July/201607-citibike-tripdata_2.csv
2016-citibike-tripdata/7_July/201607-citibike-tripdata_1.csv
2016-citibike-tri

In [53]:
#### There are two CSV files in June 2016 directory. These file will be merged and saved as CSV for further clean up

In [72]:
# Merge the 2016 June csv files 
zip_file = "2016-citibike-tripdata.zip"
csv_files_2016 = {
    "2016_1": "2016-citibike-tripdata/6_June/201606-citibike-tripdata_1.csv",
    "2016_2": "2016-citibike-tripdata/6_June/201606-citibike-tripdata_2.csv"
}

# Function to extract and read CSV file from ZIP
def read_csv_from_zip(zip_filename, csv_filename):
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        with zip_ref.open(csv_filename) as file:
            return pd.read_csv(file)

# Read CSV files and store them in a list
dfs_june_2016 = []
for key, csv_file in csv_files_2016.items():
    df = read_csv_from_zip(zip_file, csv_file)
    dfs_june_2016.append(df)

# Concatenate all DataFrames into one
june_merged_df_2016 = pd.concat(dfs_june_2016, ignore_index=True)

# Save the merged DataFrame to a CSV file
june_merged_df_2016.to_csv("2016_citibike_june_tripdata_merged.csv", index=False)

# Check the first few rows of the merged DataFrame
print(june_merged_df_2016.head())

   tripduration          starttime           stoptime  start station id  \
0          1470  6/1/2016 00:00:18  6/1/2016 00:24:48               380   
1           229  6/1/2016 00:00:20  6/1/2016 00:04:09              3092   
2           344  6/1/2016 00:00:21  6/1/2016 00:06:06               449   
3          1120  6/1/2016 00:00:28  6/1/2016 00:19:09               522   
4           229  6/1/2016 00:00:53  6/1/2016 00:04:42               335   

         start station name  start station latitude  start station longitude  \
0          W 4 St & 7 Ave S               40.734011               -74.002939   
1         Berry St & N 8 St               40.719009               -73.958525   
2           W 52 St & 9 Ave               40.764618               -73.987895   
3   E 51 St & Lexington Ave               40.757148               -73.972078   
4  Washington Pl & Broadway               40.729039               -73.994046   

   end station id         end station name  end station latitude  \


# Data Integrity

#### Data integrity is being assessed by checking for null values in all columns.The June 2013 dataset was dropped as a key analytical column contained more than 40% null values, which would significantly impact results.

In [73]:
null_percentage = june_merged_df_2016.isnull().mean() * 100
print(null_percentage)

tripduration                0.000000
starttime                   0.000000
stoptime                    0.000000
start station id            0.000000
start station name          0.000000
start station latitude      0.000000
start station longitude     0.000000
end station id              0.000000
end station name            0.000000
end station latitude        0.000000
end station longitude       0.000000
bikeid                      0.000000
usertype                    0.000000
birth year                 13.367362
gender                      0.000000
dtype: float64


In [74]:
# Checking for Duplicates 
duplicates = june_merged_df_2016[june_merged_df_2016.duplicated()]
print(duplicates)
duplicates = june_merged_df_2019[june_merged_df_2019.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [tripduration, starttime, stoptime, start station id, start station name, start station latitude, start station longitude, end station id, end station name, end station latitude, end station longitude, bikeid, usertype, birth year, gender]
Index: []
Empty DataFrame
Columns: [tripduration, starttime, stoptime, start station id, start station name, start station latitude, start station longitude, end station id, end station name, end station latitude, end station longitude, bikeid, usertype, birth year, gender]
Index: []


In [75]:
june_merged_df_2019.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,330,2019-06-01 00:00:01.5000,2019-06-01 00:05:31.7600,3602,31 Ave & 34 St,40.763154,-73.920827,3570,35 Ave & 37 St,40.755733,-73.923661,20348,Subscriber,1992,1
1,830,2019-06-01 00:00:04.2400,2019-06-01 00:13:55.1470,3054,Greene Ave & Throop Ave,40.689493,-73.942061,3781,Greene Av & Myrtle Av,40.698568,-73.918877,34007,Subscriber,1987,2
2,380,2019-06-01 00:00:06.0190,2019-06-01 00:06:26.7790,229,Great Jones St,40.727434,-73.993790,326,E 11 St & 1 Ave,40.729538,-73.984267,20587,Subscriber,1990,2
3,1155,2019-06-01 00:00:06.7760,2019-06-01 00:19:22.5380,3771,McKibbin St & Bogart St,40.706237,-73.933871,3016,Kent Ave & N 7 St,40.720368,-73.961651,33762,Subscriber,1987,1
4,1055,2019-06-01 00:00:07.5200,2019-06-01 00:17:42.5580,441,E 52 St & 2 Ave,40.756014,-73.967416,3159,W 67 St & Broadway,40.774925,-73.982666,31290,Subscriber,1973,1


In [76]:
june_merged_df_2016.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1470,6/1/2016 00:00:18,6/1/2016 00:24:48,380,W 4 St & 7 Ave S,40.734011,-74.002939,3236,W 42 St & Dyer Ave,40.758985,-73.993800,19859,Subscriber,1972.0,1
1,229,6/1/2016 00:00:20,6/1/2016 00:04:09,3092,Berry St & N 8 St,40.719009,-73.958525,3103,N 11 St & Wythe Ave,40.721533,-73.957824,16233,Subscriber,1967.0,1
2,344,6/1/2016 00:00:21,6/1/2016 00:06:06,449,W 52 St & 9 Ave,40.764618,-73.987895,469,Broadway & W 53 St,40.763441,-73.982681,22397,Subscriber,1989.0,1
3,1120,6/1/2016 00:00:28,6/1/2016 00:19:09,522,E 51 St & Lexington Ave,40.757148,-73.972078,401,Allen St & Rivington St,40.720196,-73.989978,16231,Subscriber,1991.0,1
4,229,6/1/2016 00:00:53,6/1/2016 00:04:42,335,Washington Pl & Broadway,40.729039,-73.994046,285,Broadway & E 14 St,40.734546,-73.990741,15400,Subscriber,1989.0,1


In [77]:
# Checking Data Types
june_merged_df_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460318 entries, 0 to 1460317
Data columns (total 15 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   tripduration             1460318 non-null  int64  
 1   starttime                1460318 non-null  object 
 2   stoptime                 1460318 non-null  object 
 3   start station id         1460318 non-null  int64  
 4   start station name       1460318 non-null  object 
 5   start station latitude   1460318 non-null  float64
 6   start station longitude  1460318 non-null  float64
 7   end station id           1460318 non-null  int64  
 8   end station name         1460318 non-null  object 
 9   end station latitude     1460318 non-null  float64
 10  end station longitude    1460318 non-null  float64
 11  bikeid                   1460318 non-null  int64  
 12  usertype                 1460318 non-null  object 
 13  birth year               1265112 non-null 

In [79]:
# Chaging type to date and time for start time and stop time columns  
june_merged_df_2016['starttime'] = pd.to_datetime(june_merged_df_2016['starttime'])
june_merged_df_2016['stoptime'] = pd.to_datetime(june_merged_df_2016['stoptime'])

In [67]:
june_merged_df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2125370 entries, 0 to 2125369
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start station id         int64  
 4   start station name       object 
 5   start station latitude   float64
 6   start station longitude  float64
 7   end station id           int64  
 8   end station name         object 
 9   end station latitude     float64
 10  end station longitude    float64
 11  bikeid                   int64  
 12  usertype                 object 
 13  birth year               int64  
 14  gender                   int64  
dtypes: float64(4), int64(6), object(5)
memory usage: 243.2+ MB
